In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=1024)
outputs = tf.keras.layers.Lambda(lambda x: tf.split(x, num_or_size_splits=1024, axis=1))(inputs)
while len(outputs) > 1:
    new_output = []
    for i in range(len(outputs)//2):
        new_output.append(tf.keras.layers.LeakyReLU(alpha=.01)(tf.keras.layers.Dense(units=1, activation=None, use_bias=False)(output)))
    outputs = new_output

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

In [18]:
import tensorflow as tf
import numpy as np

def create_new_model(current_shape=1024):
    current_shape = 1024
    inputs = tf.keras.Input(shape=current_shape)
    current_shape //= 2
    x = tf.keras.layers.Lambda(lambda x: tf.split(x, num_or_size_splits=current_shape, axis=1))(inputs)
    last_run = False
    while len(x) > 1:
        y = []
        for x_couple in x:
            y.append(tf.keras.layers.LeakyReLU(alpha=.01)(tf.keras.layers.Dense(units=1, activation=None, use_bias=False)(x_couple)))
        x = tf.keras.layers.Concatenate(axis=-1)(y)
        current_shape//=2
        x = tf.keras.layers.Lambda(lambda x: tf.split(x, num_or_size_splits=current_shape, axis=1))(x)

    x = tf.keras.layers.LeakyReLU(alpha=.01)(tf.keras.layers.Dense(units=1, activation=None, use_bias=False)(x[0]))
    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid", use_bias=False)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

    return model

def create_new_model(current_shape=1024):
    inputs = tf.keras.Input(shape=(current_shape,), dtype=tf.float64)
    current_shape //= 2
    x = tf.split(inputs, num_or_size_splits=current_shape, axis=-1)
    print(x)
    while len(x) > 1:
        y = []
        for i in range(len(x)//2):
            y.append(tf.concat(
                [tf.nn.leaky_relu(tf.keras.layers.Dense(units=1, activation=None, use_bias=False)(x[2*i]), alpha=.01),
                 tf.nn.leaky_relu(tf.keras.layers.Dense(units=1, activation=None, use_bias=False)(x[2*i+1]), alpha=.01)],
                axis=-1))
        x = y

    x = tf.keras.layers.LeakyReLU(alpha=.01)(tf.keras.layers.Dense(units=1, activation=None, use_bias=False)(x[0]))
    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid", use_bias=False)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-08),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.AUTO),
                  metrics=['binary_crossentropy', 'acc'])

    return model

2047
